In [1]:
from itertools import combinations
from math import gcd

In [2]:
# Define the input grid as a multiline string
input_grid = """\
.I.........................................U......
.....................9.......N........G.........U.
....................................GN............
...j.....A...S....................................
.j...o...S......t.....9...........................
..................................................
.....3...................9.....................G..
..I.....3...........................iG.x..2.......
......o...j...Sh.......x...t..........U....2......
.r...I.o.b.w.....h...............U............M..i
3...w......7......................................
......S.r.......h...................N.............
........7..r..........5.h.........................
....J.......7B......x...O..........y..............
......I...J..a..x..............O.......H..........
w..Fj..............t.O................H......2....
........a..m..........y...O.4.............l.......
.....Jm....................4......................
..m.......................2.Ny..............M.H...
.......a................X.....5k.....M.........H..
.........F...........A...................M.K......
.......7...........m....t...y.........K...........
....i.....................B.......................
.................5................6...............
........b..T................ABK....i..............
....................s..................K..........
.....w...J.............s.................W......n.
..............F........X...8......................
...........4..................s...........W....f..
.........g.....so.B.9..Y........4............6...f
..................................................
b......1....................8..................W..
Lb...g.R..0...................Y................l..
................................n...........f...l.
..........g..............n8.....k......6..........
.....R....A0.......................n..........W..l
...............................d..............6...
........................k..................D......
...0............X......8.k........F...............
....a....L..............Y.................D.......
.........1...L.........u...D.......Yd.............
.............................d...u................
........R...................................D....X
............L.g.0.................................
..................................................
..............................T...................
........................T.........................
....................u..........T..................
....1...................u.........................
..................R...................d..........."""

In [3]:
def parse_grid(grid_str):
    """
    Parses the input grid string into a 2D list.
    Each sublist represents a row in the grid.
    """
    grid = []
    for line in grid_str.strip().split('\n'):
        grid.append(line.rstrip())  # Remove trailing spaces
    return grid

def get_antenna_positions(grid):
    """
    Scans the grid and groups antenna positions by their frequency.
    Returns a dictionary with frequency as keys and lists of (x, y) tuples as values.
    """
    antennas = {}
    for y, row in enumerate(grid):
        for x, char in enumerate(row):
            if char != '.':
                if char not in antennas:
                    antennas[char] = []
                antennas[char].append( (x, y) )
    return antennas

def calculate_antinodes_part1(antennas, grid_width, grid_height):
    """
    Calculates antinode positions based on Part One rules.
    An antinode occurs at points where one antenna is twice as far as the other.
    Returns a set of (x, y) tuples representing antinode positions.
    """
    antinodes = set()
    for freq, positions in antennas.items():
        if len(positions) < 2:
            continue  # Need at least two antennas to form antinodes
        # Generate all unique pairs
        for p1, p2 in combinations(positions, 2):
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            # Antinode 1: p1 - (dx, dy)
            a1 = (p1[0] - dx, p1[1] - dy)
            if 0 <= a1[0] < grid_width and 0 <= a1[1] < grid_height:
                antinodes.add(a1)
            # Antinode 2: p2 + (dx, dy)
            a2 = (p2[0] + dx, p2[1] + dy)
            if 0 <= a2[0] < grid_width and 0 <= a2[1] < grid_height:
                antinodes.add(a2)
    return antinodes

def calculate_antinodes_part2(antennas, grid_width, grid_height):
    """
    Calculates antinode positions based on Part Two rules.
    An antinode occurs at any grid position exactly in line with at least two antennas of the same frequency.
    Returns a set of (x, y) tuples representing antinode positions.
    """
    antinodes = set()
    for freq, positions in antennas.items():
        if len(positions) < 2:
            continue  # Need at least two antennas to form antinodes
        # Generate all unique pairs
        for p1, p2 in combinations(positions, 2):
            x1, y1 = p1
            x2, y2 = p2
            dx = x2 - x1
            dy = y2 - y1
            if dx == 0 and dy == 0:
                continue  # Same position, skip to avoid infinite loop
            # Compute the step increments by reducing dx and dy by their GCD
            step_gcd = gcd(dx, dy)
            if step_gcd == 0:
                continue  # Avoid division by zero
            step_x = dx // step_gcd
            step_y = dy // step_gcd
            # To cover the entire line, extend in both directions
            # Extend in the negative direction from p1
            current_x, current_y = x1, y1
            while True:
                current_x -= step_x
                current_y -= step_y
                if 0 <= current_x < grid_width and 0 <= current_y < grid_height:
                    antinodes.add( (current_x, current_y) )
                else:
                    break
            # Extend in the positive direction from p2
            current_x, current_y = x2, y2
            while True:
                current_x += step_x
                current_y += step_y
                if 0 <= current_x < grid_width and 0 <= current_y < grid_height:
                    antinodes.add( (current_x, current_y) )
                else:
                    break
            # Now, add all positions between p1 and p2 (inclusive)
            # Starting from p1
            current_x, current_y = x1, y1
            while True:
                antinodes.add( (current_x, current_y) )
                if current_x == x2 and current_y == y2:
                    break
                current_x += step_x
                current_y += step_y
                if not (0 <= current_x < grid_width and 0 <= current_y < grid_height):
                    break
    return antinodes

def count_unique_antinodes_part1(grid_str):
    """
    Orchestrates the parsing, antenna identification, and antinode calculation for Part One.
    Returns the total number of unique antinode positions for Part One.
    """
    grid = parse_grid(grid_str)
    grid_height = len(grid)
    grid_width = max(len(row) for row in grid) if grid else 0
    antennas = get_antenna_positions(grid)
    antinodes = calculate_antinodes_part1(antennas, grid_width, grid_height)
    return len(antinodes)

def count_unique_antinodes_part2(grid_str):
    """
    Orchestrates the parsing, antenna identification, and antinode calculation for Part Two.
    Returns the total number of unique antinode positions for Part Two.
    """
    grid = parse_grid(grid_str)
    grid_height = len(grid)
    grid_width = max(len(row) for row in grid) if grid else 0
    antennas = get_antenna_positions(grid)
    antinodes = calculate_antinodes_part2(antennas, grid_width, grid_height)
    return len(antinodes)

def main():
    """
    Main function to execute Part One and Part Two calculations.
    """
    total_antinodes_part1 = count_unique_antinodes_part1(input_grid)
    print(f"Part One - Total unique antinode locations: {total_antinodes_part1}")
    
    total_antinodes_part2 = count_unique_antinodes_part2(input_grid)
    print(f"Part Two - Total unique antinode locations: {total_antinodes_part2}")

if __name__ == "__main__":
    main()

Part One - Total unique antinode locations: 348
Part Two - Total unique antinode locations: 1221
